In [36]:
import cpmpy as cp

In [37]:
n = 9 # number of blocks
k = 3 # number of piles

start = [2, 0, 9, 6, 3, 1, 5, 4, 0]
goal =  [3, 8, 4, 5, 0, 9, 0, 0, 1]

# n = 5
# k = 3
# start = [5, 0, 2, 0, 4]
# goal = [2, 3, 0, 5, 0]


In [38]:
# import matplotlib.pyplot as plt
def visualize(state):
    state = [x - 1 for x in state]
    for next_idx in range(len(state)):
        if next_idx in state: continue
        tower = [next_idx+1]
        while next_idx >= 0:
            x = state[next_idx]
            tower.append(x+1)
            next_idx = x
        print(tower, end="\t")
    print()
visualize([2, 3, 0, 0, 4])

[1, 2, 3, 0]	[5, 4, 0]	


In [41]:

import cpmpy as cp
from cpmpy.tools.explain import mus
FROM, TO = 0,1
nCubes, horizon = n + 1, n * k + 1  # nCubes includes the dummy cube 0

state = cp.intvar(0,n,   shape=(nCubes, horizon), name="state")
count = cp.intvar(0,k,   shape=(nCubes, horizon), name="count")
done = cp.boolvar(       shape=horizon, name="done")
move = cp.intvar(0,n+1,  shape=(horizon,2), name="move")
locked = cp.boolvar(     shape=(nCubes, horizon), name="locked")

model = cp.Model()
model.minimize(horizon - cp.sum(done))

# dummy block is always on top
model += state[0,:] == 0

# define start state
model += state[1:, 0] == start

# end state (redundant constraint)
model += state[1:,-1] == goal

# define doneness
for t in range(horizon):
    model += cp.all(state[1:, t] == goal) == done[t]
model += cp.Increasing(done)


# define moves
for t in range(horizon):
    model += move[t,TO] == state[:,t][move[t,FROM]]

for t in range(1, horizon):
    is_done = done[t-1]

    model +=  is_done.implies(cp.all(state[1:, t-1] == state[1:,t])) # don't move
    model += (~is_done).implies(cp.all([(state[b, t-1] != state[b, t]) == (b == move[t,FROM]) for b in range(nCubes)]))
    model += (~is_done).implies(count[:, t-1][move[t-1, FROM]] == 0)

    # no more moves once finished
    model += is_done.implies(move[t,0] == 0)




# count nb of times a block occurs
model += count[0, :] >= 1
model += count[1:, :] <= 1

for t in range(horizon):
    model += cp.GlobalCardinalityCount(state[1:, t], list(range(0,nCubes)), count[:, t], closed=True)


######################################
if model.solve() is False:
    print("UNSAT")
    print("Finding MUS")
    for c in mus(model.constraints):
        print("-", c)

else:
    print(model.status())
    obj = model.objective_value()
    print(model.objective_value())

    for s in state.value().T:
        print(s[1:])
        visualize(s[1:])
        print("-"*30)

ExitStatus.OPTIMAL (1.9717630000000002 seconds)
15
[2 0 9 6 3 1 5 4 0]
[7, 5, 3, 9, 0]	[8, 4, 6, 1, 2, 0]	
------------------------------
[2 0 9 6 3 1 0 4 0]
[5, 3, 9, 0]	[7, 0]	[8, 4, 6, 1, 2, 0]	
------------------------------
[2 0 9 6 7 1 0 4 0]
[3, 9, 0]	[5, 7, 0]	[8, 4, 6, 1, 2, 0]	
------------------------------
[2 0 9 6 7 1 0 3 0]
[4, 6, 1, 2, 0]	[5, 7, 0]	[8, 3, 9, 0]	
------------------------------
[2 0 9 5 7 1 0 3 0]
[4, 5, 7, 0]	[6, 1, 2, 0]	[8, 3, 9, 0]	
------------------------------
[2 0 9 5 7 1 0 6 0]
[3, 9, 0]	[4, 5, 7, 0]	[8, 6, 1, 2, 0]	
------------------------------
[2 0 4 5 7 1 0 6 0]
[3, 4, 5, 7, 0]	[8, 6, 1, 2, 0]	[9, 0]	
------------------------------
[2 0 4 5 7 1 0 6 3]
[8, 6, 1, 2, 0]	[9, 3, 4, 5, 7, 0]	
------------------------------
[2 0 4 5 7 1 0 0 3]
[6, 1, 2, 0]	[8, 0]	[9, 3, 4, 5, 7, 0]	
------------------------------
[2 0 4 5 7 8 0 0 3]
[1, 2, 0]	[6, 8, 0]	[9, 3, 4, 5, 7, 0]	
------------------------------
[2 0 4 5 7 8 0 0 6]
[1, 2, 0]	[3, 4, 5, 7, 0]	[

In [40]:
print(model)

Constraints:
    [state[0,0] == 0 state[0,1] == 0 state[0,2] == 0 state[0,3] == 0
 state[0,4] == 0 state[0,5] == 0 state[0,6] == 0 state[0,7] == 0
 state[0,8] == 0 state[0,9] == 0 state[0,10] == 0 state[0,11] == 0
 state[0,12] == 0 state[0,13] == 0 state[0,14] == 0 state[0,15] == 0
 state[0,16] == 0 state[0,17] == 0 state[0,18] == 0 state[0,19] == 0
 state[0,20] == 0 state[0,21] == 0 state[0,22] == 0 state[0,23] == 0
 state[0,24] == 0 state[0,25] == 0 state[0,26] == 0 state[0,27] == 0]
    [state[1,0] == 2 state[2,0] == 0 state[3,0] == 9 state[4,0] == 6
 state[5,0] == 3 state[6,0] == 1 state[7,0] == 5 state[8,0] == 4
 state[9,0] == 0]
    [state[1,27] == 3 state[2,27] == 8 state[3,27] == 4 state[4,27] == 5
 state[5,27] == 0 state[6,27] == 9 state[7,27] == 0 state[8,27] == 0
 state[9,27] == 1]
    (and([state[1,0] == 3, state[2,0] == 8, state[3,0] == 4, state[4,0] == 5, state[5,0] == 0, state[6,0] == 9, state[7,0] == 0, state[8,0] == 0, state[9,0] == 1])) == (done[0])
    (and([state[1,